## Qiskit Aqua Gradient Framework

### First Order Gradients

#### Gradients w.r.t. Measurement Operator Coefficients

#### Gradients w.r.t. Measurement Quantum State Parameters

##### Parameter Shift Parameters

##### Linear Combination of Unitaries Parameters

#### Natural Gradient

### Second Order Gradients

#### Gradients w.r.t. Measurement Operator Coefficients

#### Gradients w.r.t. Measurement Quantum State Parameters

##### Parameter Shift Parameters

##### Linear Combination of Unitaries Parameters

### QFI

#### Full QFI
Using linear combination of unitaries

#### Block-diagonal Approximation
Without working qubits
Requires unrolling into Pauli rotations and unparameterized Gates

#### Diagonal Approximation
Without working qubits
Requires unrolling into Pauli rotations and unparameterized Gates

### Application Examples

#### VQE with first and second order gradient based optimization

#### Variational Quantum Imaginary Time Evolution (VarQITE)


#### QBMs? - see if time allows